# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [1]:
!pip install wandb -qU

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [3]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [5]:
diamonds.shape

(53940, 10)

In [6]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

In [7]:
print(X)

       carat        cut color clarity  depth  table     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0  5.75  5.76  3.50
53936   0.72       Good     D     SI1   63.1   55.0  5.69  5.75  3.61
53937   0.70  Very Good     D     SI1   62.8   60.0  5.66  5.68  3.56
53938   0.86    Premium     H     SI2   61.0   58.0  6.15  6.12  3.74
53939   0.75      Ideal     D     SI2   62.2   55.0  5.83  5.87  3.64

[53940 rows x 9 columns]


### Split the data and train a model

In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [9]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:39:53] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [10]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
mse = mean_squared_error(y_test, predictions, squared=True)
print(f"RMSE: {rmse} , MSE = {mse}")

RMSE: 532.8838153117543 , MSE = 283965.1606212119


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:39:54] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


### Incorporate validation

In [11]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [12]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[10]	train-rmse:550.99470	validation-rmse:571.16640
[20]	train-rmse:491.51435	validation-rmse:544.08058
[30]	train-rmse:464.38845	validation-rmse:537.01895


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:39:54] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [13]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:39:55] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[103]	train-rmse:375.44970	validation-rmse:532.50220


In [14]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


In [15]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [16]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [17]:
# Log in to your W&B account
import wandb

wandb.login()

wandb: Currently logged in as: jeongyoonlee2024 (northwestern-jeongyoon). Use `wandb login --relogin` to force relogin


True

In [18]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project_XGBoost_experiment1",
    job_type = "train_model"
)

sweep_config = {
  "method" : "random", # Choose any method for hyperparameter tuning: grid search, random search, bayesian search
  "parameters" : {
    "learning_rate" :{"values" : [0.001, 0.01, 0.1]},
    "gamma" :{"values" : [0.001, 0.01, 0.1]},
    "early_stopping_rounds" :{
      "values" : [10, 30, 50]
    },
    "max_depth": {
      "values": [3, 5, 7]
        }
  }
}



In [19]:
sweep_id = wandb.sweep(sweep_config, project="my-awesome-project_XGBoost_experiment1")

Create sweep with ID: 6fptewy1
Sweep URL: https://wandb.ai/northwestern-jeongyoon/my-awesome-project_XGBoost_experiment1/sweeps/6fptewy1


In [20]:
def train():
    # init()
    run = wandb.init(job_type = "train_model")

    # Hyperparameters
    config = run.config

    # Define the model
    params = {
        'objective': 'reg:squarederror',
        'gamma' : config.gamma,
        'learning_rate': config.learning_rate,
        'max_depth': int(config.max_depth),
        'early_stopping_rounds' : int(config.early_stopping_rounds),
        'eval_metric': 'rmse'
    }

    # Train the model
    model = xgb.train(params, dtrain)

    # Evaluate the model
    predictions = model.predict(dtest)
    rmse = np.sqrt(mean_squared_error(y_test, predictions, squared = False))
    mse = np.sqrt(mean_squared_error(y_test, predictions, squared = True))

    # Log metrics
    wandb.log({'rmse': rmse , 'mse' : mse})

    run.finish()


In [21]:
wandb.agent(sweep_id, train)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 47ny7wmz with config:
wandb: 	early_stopping_rounds: 30
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 5


mse,▁
rmse,▁
mse,1594.07739
rmse,39.9259


Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 286, in check_stop_status
    self._loop_check_status(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 224, in _loop_check_status
    local_handle = request()
Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/interface/interface.py", line 840, in deliver_stop_status
Exception in thread IntMsgThr    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner

mse,▁
rmse,▁
mse,3949.00773
rmse,62.84113


wandb: Agent Starting Run: qif3b8ml with config:
wandb: 	early_stopping_rounds: 30
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 7


...

wandb: Agent Starting Run: 8dagc01t with config:
wandb: 	early_stopping_rounds: 30
wandb: 	gamma: 0.001
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 7


mse,▁
rmse,▁
mse,3620.29134
rmse,60.16886


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7967383bbb80>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

### Description of the experiment

Best model :
Early stopping: 30
Gamma: 0.001
Learning Rate: 0.1
Max_depth : 7

rmse :39.052 , mse: 1525.078

## Hyper-parameter Setting Variarity:
Used Random Search to check the combination between hyperparameters.


When the gamma value is big, it can prevent overfitting.
When the max_depth value gets bigger, the model can have more information but has the possibility of overfitting.

For example, during the experiment, it comes out with a big rmse value when the max_depth = 7, and the learning rate is quite high (i.e., 0.1).




They have a good result which is that rmse values are less than 40 when their hyperparameter combination with gamma and learning rate is opposite. (when the gamma value is small, the learning rate is large, vice versa.)


To prevent overfitting since we have lots of experiments, I used early stopping rounds so that the model can stop training if they are not learning more.

I believe if I use several numbers of n_estimator, the model performance will get better.

The experience that I had with W&B was really insightful, I can check the best model in the easy-visualized plot on the website and check the difference between hyperparameters.
